In [ ]:
#Imports
import requests
import pandas as pd
import certifi
from decimal import Decimal
import pyodbc
from sqlalchemy import engine
from datetime import datetime


#Prototypisches Tool zum Auslesen Transaktionaler Mandantendaten auf der Blockchain
#Author: Nils Taglieber
#Datum: 04.04.2022

#Ausgabe mit Aufforderung zur Aktionseingabe
print("""Um Transaktionen von Etherscan abzufragen und in der Zentralen Datenbank zu speichern schreiben sie: Etherscan
Um Transaktionen für eine Transaktionsadresse in Excel ausgeben zu lassen schreiben sie: Excel
Um das Programm zu beenden schreiben sie: Ende""")

eingabe = input("Eingabemöglichkeiten: Etherscan; Excel; Ende")

#Eingabeschleife zum Aufruf der gewünschten Programmabschnitte
while eingabe != "Ende":
    transaktionsaddresse = input("Für welche Transaktionsaddresse soll das Programm Ausgeführt werden ?")
    if eingabe == "Etherscan":
        store_data(transaktionsaddresse)
    elif eingabe == "Excel":
        get_data(transaktionsaddresse) 
    else:
        print("Unzulässige Eingabe")
    eingabe = input("Eingabemöglichkeiten: Etherscan; Excel; Ende")
    if eingabe == "Ende":
        break
    transaktionsaddresse = input("Für welche Transaktionsaddresse soll das Programm Ausgeführt werden ?")

In [ ]:
def store_data(address):
    address                                                     #Transaktionsaddresse
    API_key = "FRDWSCVDMQ1E85TPDMUBCNH86ZCP55MA46"              #Api Key -> Notwendig für zugriff, über Etherscan erstellbar
    number_transactions = "1000"                                #Zahl der Transaktionen bis zu 10000
    order  = "desc"                                             #Absteigend/Aufsteigend Anordnung der Transaktionen

    #Connect to Etherscan Api
    url = "https://api.etherscan.io/api?module=account&action=txlist&address={}&startblock=0&endblock=99999999&page={}&offset={}&sort={}&apikey={}".format(address,1,number_transactions,order,API_key)

    #Data in Json Format umwandeln und Wesentliche Daten Extrahieren
    data = requests.get(url, verify=certifi.where())
    jsondata = data.json() #dict

    #Aufruf der get_frame Funktion mit nötigen Parametern
    df = get_frame(jsondata['result'], address)                      
    df = df.iloc[:, [1,0,2,3,4,5,6,7,8,9,10,11,12]]            #Reorganisation der Dataframespalten

    #Dataframe to SQL
    #Connection Daten
    server = 'de-20hjpf0v4bt8.atrema.deloitte.com'
    database= 'NilsTestDB'
    username = 'Nils'
    password = 'Ha1nfeld_76835'

    #Connect to DB
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()                                     #cursor zum navigieren

    #Daten in SQL Tabelle Schreiben
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO Eth_Data1 VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)", row.Date, row.blockNumber, row.transaction_hash, row.blockHash,row.tx_from, row.tx_to, row.eth_value,
                        row.fee, row.worked, row.contractAddress, row.confirmations, row.confirmed, row.out)

    cnxn.commit

    cursor.close()
    cnxn.close()

In [ ]:
#Daten aus Datenbank mittels Stored Procedures abfragen und in Excel Sheet schreiben
#Doppelte einträge in Datenbank durch Stored Procedure löschen

def get_data(address):
    #Connection Daten
    server = 'de-20hjpf0v4bt8.atrema.deloitte.com'
    database= 'NilsTestDB'
    username = 'Nils'
    password = 'Ha1nfeld_76835'

    #Connect to DB
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()                                  #cursor zum navigieren

    #prepare stored procedure
    params = address

    #Stored Procedure zum Auslesen der Transaktionsdaten für eine bestimmte Transaktionsaddresse
    trans_proc = "Exec [dbo].[getTransactions] @inputAddress = ?"
    #Stored Procedure zum löschen doppelter Einträge
    del_proc = "Exec [dbo].[Delete_Duplicates]"

    #Ausführung beider Stored Procedures
    cursor.execute(del_proc)
    cnxn.commit
    cursor.execute(trans_proc, params)
    cnxn.commit

    #Abruf aller gefundener Einträge
    row = cursor.fetchall()

    #erzeugung eines Leeren objekts um Daten in guter Struktur zu speichern
    insertObject = []

    #Abfrage der Tabellenüberschriften
    columnNames = [column[0] for column in cursor.description]

    #zusammenfügen von Tabellenüberschriften und Spalteninhalten
    for record in row:
        insertObject.append( dict( zip( columnNames , record ) ) )

    #Data Frame aus insertObject erzegen
    SQL_DATA = pd.DataFrame(insertObject)
    
    #Dataframe in excel übertragen
    excel_datei = str(input("bitte name einer vorhandenen oder zu erstellenden Excel Datei angeben"))
    SQL_DATA.to_excel(excel_datei, index = False, header = True, sheet_name='new')

In [ ]:
#Frame erzeugen
#list wanted_data wird übergeben
def get_frame(wanted_data,address):
    #Frame wird erzeugt und Spaltennamen werden eingerichtet
    frame = pd.DataFrame(wanted_data)
    frame['out'] = False
    frame['confirmed'] = False
    frame.columns = ['blockNumber', 'Date', 'hash', 'nonce', 'blockHash','transactionIndex','from','to', 'value', 'gas', 'gasPrice', 'isError', 'txreceipt_status', 'input', 'contractAddress','cumulativeGasUsed','gasUsed', 'confirmations', 'confirmed', 'out']

    #Frame Wird mit neuen Daten befüllt, Spaltennamen umbenannte, Daten manipuliert und neue Spalten Hinzugefügt
    #Schleife für jede Zeile des Dataframes
    for n, transaction in enumerate(wanted_data):

        get_fee = Decimal(transaction.get("gasPrice")) * Decimal(transaction.get("gasUsed")) #berechnung gas fees aus gasPrice und gasUsed
        fee = Decimal(get_fee)/Decimal("1000000000000000000")                                #Umrechnung in Dezimalzahl                

        value = transaction.get('value')
        new_eth_value = Decimal(value)/Decimal("1000000000000000000")                        #Transactionsvalue wird in Dezimalzahl umgerechnet

        confirmations = transaction.get("confirmations")                                     #Gibt an ob die Transaktion bereits bestätigt ist(ab 16 confirmations)
        if int(confirmations) >= 16:
            confirmed = True
        else:
            confirmed = False

        tx_from = str(transaction.get('from'))                                              
        if tx_from.lower() == address.lower():                                               #Speichert ob die Transaktion eingehend oder Ausgehend ist ->Senderaddresse entspricht abfrageadresse = Sendung
            out = True
        else:
            out = False

        
        date = int(transaction.get("timeStamp"))
        dt_object = datetime.fromtimestamp(date)

        #Daten werden in Frame geschrieben
        frame.at[n,'out'] = out
        frame.at[n,'confirmed'] = confirmed
        frame.at[n,'gas'] = float(fee)
        frame.at[n,'value'] = float(new_eth_value)
        frame.at[n,'Date'] = dt_object
    
    #Umbenennung der Spaltennamen, da vor schleife nicht möglich
    frame.columns = ['blockNumber', 'Date', 'transaction_hash', 'nonce', 'blockHash','transactionIndex','tx_from','tx_to', 'eth_value', 'fee', 'gasPrice', 'isError', 'worked', 'input', 'contractAddress','cumulativeGasUsed','gasUsed', 'confirmations','confirmed','out']

    #Entfernung überflüssiger Spalten
    frame = frame.drop(['nonce', 'transactionIndex', 'gasPrice', 'isError','input', 'cumulativeGasUsed', 'gasUsed'],1 )
    
    return frame #rückgabe fertiger Frame